In [1]:
import torch
print(torch.__version__)  #注意是双下划线

1.4.0


In [8]:
a=torch.ones((2,5,4))
a.shape

torch.Size([2, 5, 4])

In [9]:
a

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [10]:
a.sum(axis=1)  #axis=1，消除Size([2, 5, 4])中索引为1的那个维度，将5消除，变成2*4的矩阵

tensor([[5., 5., 5., 5.],
        [5., 5., 5., 5.]])

# 自动求导

In [28]:
x= torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

计算y关于x的梯度之前，需要一个地方储存梯度

In [29]:
x.requires_grad_(True) #等价于 x= torch.arange（4.0，requires_grad=True）

tensor([0., 1., 2., 3.], requires_grad=True)

In [30]:
y = 2*torch.dot(x,x) #x的内积乘2
y

tensor(28., grad_fn=<MulBackward0>)

通过反向传播计算y关于x每个分量的梯度

In [31]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [32]:
x.grad == 4*x # y关于x的倒数为4*x

tensor([True, True, True, True])

In [34]:
# 在默认情况下，pytorch会积累梯度, x.grad.zero_()清除梯度
x.grad.zero_()
y = x.sum()
y.backward()
x.grad


tensor([1., 1., 1., 1.])

如果没有x.grad.zero_()梯度清0,会将两次的梯度相加tensor([ 0.,  4.,  8., 12.]) + tensor([1., 1., 1., 1.])
= tensor([1., 5., 9., 13.])

# 1.线性回归
 1.1 生成数据集

In [2]:
%matplotlib inline 
import random  #随机数
import torch 
from d2l import torch as d2l

In [2]:
def synthetic_data(w, b, num_examples):  
    """生成 y=Xw + b + 噪声"""
    X = torch.normal(0, 1, (num_examples, len(w))) # 正态分布（均值为0，标准差为1），num_examples：n个样本，w：列数
    y = torch.matmul(X, w) + b # 矩阵相乘
    y += torch.normal(0, 0.01, y.shape) # 加入噪声项
    # 得到的y为行向量的形式，为了使其变为一列的形式需要进行reshape
    return X, y.reshape((-1, 1))  # x,y从行向量变为列向量

true_w=torch.tensor([2,-3.4])
true_b=4.2
features, labels = synthetic_data(true_w,true_b,1000)
# features, labels

1.2 读取数据
定义一个data_iter函数，该函数接受批量大小，特征矩阵和标签向量作为输入，生成大小为batch_size的小批量

In [3]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples)) # indices是一个序列
    # 这些样本是随机读出的，没有特定的顺序
    random.shuffle(indices)  #将下标随机打乱
    for i in range(0, num_examples, batch_size): # 从0开始到最后，步长batch_size
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)])
#         print(batch_indices)
#         print("----------")
        # min(i+batch_size,num_examples) 在i+batch_size和num_examples中取一个最小数，
        # 防止最后一个batch数据量不够，从而报错
        yield features[batch_indices],labels[batch_indices]
        
batch_size = 10
# for X,y in data_iter(batch_size, features, labels):
#     print(X,'\n',y)
#     print("+++++++++++++++++")

1.3 初始化模型参数
在使用小批量梯度优化我们的模型参数之前，我们需要首先初始化模型参数，
有了初始化的模型参数，我们就可以更新这些参数，直到这些参数足够拟合我们的数据。
因为每次更新参数的时候都需要计算损失函数关于模型参数的梯度，
以便向减小损失函数的方向更新模型参数，所以我们引入自动微分requires_grad来计算梯度。

In [4]:
# 初始化模型参数
w = torch.normal(0,0.01,(2,1),requires_grad=True) # 均值为0，方差为0.01的正态分布
b = torch.zeros(1,requires_grad=True)
w,b

(tensor([[0.0051],
         [0.0039]], requires_grad=True),
 tensor([0.], requires_grad=True))

1.4 定义模型
我们需要定义一个模型，将模型的输入和参数同模型的输出关联起来

In [5]:
def linreg(X,w,b):
    # 线性回归模型
    return torch.matmul(X,w)+b

1.5 定义损失函数
均方损失函数

In [6]:
def squared_loss(y_hat,y):
    return (y_hat-y.reshape(y_hat.shape))**2 / 2 #没有计算均值

1.6 定义优化算法

In [7]:
def sgd(params, lr, batch_size):# params：是list，包含参数b和w，lr：学习率，
    #小批量梯度下降
    with torch.no_grad(): #更新的时候不采用梯度计算
        for param in params:   # param： w或者b
            param -= lr*param.grad/batch_size # 除以batch_size，相当于计算均值
            param.grad.zero_() # 梯度清0


1.7 训练

In [8]:
lr = 0.03  # 学习率
num_epochs = 3  # 整个数据扫三遍
net  = linreg    # 模型
loss = squared_loss  #损失函数


for epoch in range(num_epochs):
    for X, y in data_iter(batch_size,features,labels):
        l = loss(net(X,w,b),y) # x和y的小批量损失
        # 因为l的形状是（batch_size，1），而不是标量
        #  计算关于【w,b】的梯度
        l.sum().backward() # 求和之后算梯度
        sgd([w,b],lr,batch_size) # 使用参数的梯度更新w,b
        
    with torch.no_grad():  # 打印时候，不用梯度
        train_l = squared_loss(net(features,w,b),labels)
        # print(train_l)
        print(f'epoch{epoch+1},loss{float(train_l.mean()):f}')

epoch1,loss0.047493
epoch2,loss0.000240
epoch3,loss0.000055


1.8 比较真实参数和通过训练学到的参数来评估训练的成功程度

In [11]:
print(f'w的估计误差：{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差：{true_b - b}')

w的估计误差：tensor([ 0.0007, -0.0007], grad_fn=<SubBackward0>)
b的估计误差：tensor([0.0005], grad_fn=<RsubBackward1>)


# 线性回归完整实现

In [3]:
import torch
import random

def synthetic_data(w, b, num_examples):
    """生成 y=Xw + b + 噪声"""
    X = torch.normal(0, 1, (num_examples, len(w))) # 正态分布（均值为0，标准差为1）
    y = torch.matmul(X, w) + b # 矩阵相乘
    y += torch.normal(0, 0.01, y.shape) # 加入噪声项
    # 得到的y为行向量的形式，为了使其变为一列的形式需要进行reshape
    return X, y.reshape((-1, 1))

def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读出的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)])
        yield features[batch_indices],labels[batch_indices]

def linear(X,w,b):
    """定义模型"""
    return torch.matmul(X,w)+b

def squared_loss(y_hat,y):
    return (y_hat-y.reshape(y_hat.shape))**2/2

def sgd(params, lr, batch_size):
    """小批量梯度下降"""
    with torch.no_grad():
        for param in params:  # 参数b和w
            param -= lr*param.grad/batch_size
            param.grad.zero_()

if __name__ == '__main__':

    true_w = torch.tensor([2, -3.4])
    true_b = 4.2
    features, labels = synthetic_data(true_w,true_b,1000) # 生成数据集
    # 初始化模型参数
    w = torch.normal(0,0.01,(2,1),requires_grad=True)
    b = torch.zeros(1,requires_grad=True)

    # 定义超参数
    lr = 0.03
    num_epochs = 3
    batch_size = 10
    net = linear
    loss = squared_loss

    for epoch in range(num_epochs):
        for X, y in data_iter(batch_size,features,labels):
            y_hat = linear(X,w,b)
            loss = squared_loss(y_hat,y)
            loss.sum().backward() # 进行反向传播得到梯度
            sgd((w,b),lr,batch_size)
        with torch.no_grad():
            train_l = squared_loss(net(features,w,b),labels)
            # print(train_l)
            print(f'epoch{epoch+1},loss{float(train_l.mean()):f}')

            
"""
output：
epoch1,loss0.362755
epoch2,loss0.008698
epoch3,loss0.000259
"""

epoch1,loss0.033085
epoch2,loss0.000120
epoch3,loss0.000054


'\noutput：\nepoch1,loss0.362755\nepoch2,loss0.008698\nepoch3,loss0.000259\n'

 # 2.线性回归简洁实现

In [4]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l


In [5]:
def synthetic_data(w, b, num_examples):  
    """生成 y=Xw + b + 噪声"""
    X = torch.normal(0, 1, (num_examples, len(w))) # 正态分布（均值为0，标准差为1），num_examples：n个样本，w：列数
    y = torch.matmul(X, w) + b # 矩阵相乘
    y += torch.normal(0, 0.01, y.shape) # 加入噪声项
    # 得到的y为行向量的形式，为了使其变为一列的形式需要进行reshape
    return X, y.reshape((-1, 1))  # x,y从行向量变为列向量

true_w=torch.tensor([2,-3.4])
true_b=4.2
features, labels = synthetic_data(true_w,true_b,1000)

In [6]:
def load_array(data_arrays, batch_size, is_train=True):
    """构建一个pytorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
    # shuffle= is_train=true：随机打乱；
batch_size = 10
data_iter = load_array((features,labels),batch_size)
next(iter(data_iter))

[tensor([[-0.0276,  1.3525],
         [-0.5965, -2.0344],
         [ 1.3304, -0.7538],
         [ 0.4916,  0.0371],
         [ 1.0540,  1.2262],
         [ 1.1513,  1.7523],
         [-0.3448, -0.3764],
         [-0.2247,  0.0912],
         [ 0.7380, -0.6897],
         [ 0.8165,  0.5144]]),
 tensor([[-0.4609],
         [ 9.9160],
         [ 9.4162],
         [ 5.0823],
         [ 2.1411],
         [ 0.5540],
         [ 4.8031],
         [ 3.4300],
         [ 8.0203],
         [ 4.0828]])]

from torch.utils import data
dataset   = data.TensorDataset(features, labels)
data_iter = data.DataLoader(dataset, batch_size, shuffle=True)

for i, data in enumerate (data_iter):
    x_data, label = data
    print(i,x_data,label)

In [7]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))  # pytorch 的word文档


In [8]:
#更换权重
net[0].weight.data.normal_(0,0.01) 
# data是真实的data；data.normal_(0,0.01)：将data换成(0,0.01)的正态分布。

# 更换偏置
net[0].bias.data.fill_(0)
# data是真实的data；fill_(0)：将data换成0。

tensor([0.])

In [9]:
# 均方误差，也称L2范数
loss = nn.MSELoss()

In [10]:
#实例化SGD实例
trainer = torch.optim.SGD(net.parameters(),0.03)

In [11]:
num_epoch = 3
for epoch in range(num_epoch):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step() # 优化
    l = loss(net(features),labels)
    print(f'epoch{epoch+1},loss{l:f}')


epoch1,loss0.000235
epoch2,loss0.000096
epoch3,loss0.000094


# 一次性简洁实现

In [99]:
import numpy as np
import torch
from torch.utils import data

def synthetic_data(w, b, num_examples):  
    """生成 y=Xw + b + 噪声"""
    X = torch.normal(0, 1, (num_examples, len(w))) # 正态分布（均值为0，标准差为1），num_examples：n个样本，w：列数
    y = torch.matmul(X, w) + b # 矩阵相乘
    y += torch.normal(0, 0.01, y.shape) # 加入噪声项
    # 得到的y为行向量的形式，为了使其变为一列的形式需要进行reshape
    return X, y.reshape((-1, 1))  # x,y从行向量变为列向量

true_w=torch.tensor([2,-3.4])
true_b=4.2
features, labels = synthetic_data(true_w,true_b,1000)

def load_array(data_arrays, batch_size, is_train=True):
    """构建一个pytorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
    # shuffle= is_train=true：随机打乱；
batch_size = 10
data_iter = load_array((features,labels),batch_size)
# next(iter(data_iter))

from torch import nn
net = nn.Sequential(nn.Linear(2,1))   # pytorch 的word文档

# net[0].weight.data,net[0].bias.data

#更换权重
net[0].weight.data.normal_(0,0.01) 
# data是真实的data；data.normal_(0,0.01)：将data换成(0,0.01)的正态分布。
# 更换偏置
net[0].bias.data.fill_(0)
# data是真实的data；fill_(0)：将data换成0。

# 均方误差，也称L2范数
loss = nn.MSELoss()

#实例化SGD实例
trainer = torch.optim.SGD(net.parameters(),0.03)

num_epoch = 3
for epoch in range(num_epoch):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()  # 梯度清零
        l.backward()    # 得到梯度
        trainer.step() # 模型更新
    l = loss(net(features),labels)
    print(f'epoch{epoch+1},loss{l:f}')
    

epoch1,loss0.000247
epoch2,loss0.000106
epoch3,loss0.000106
